<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci553-data-mining-sp24/blob/main/assignment-6/notebooks/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation & Setup

In [ ]:
%pip install pyspark ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e799adc1437f6b12d4d9fc10fa2cd660e2b8e1c55c7a22806a8743c50e2f5351
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Imports

In [ ]:
import os
import sys
import json
import pandas as pd
from itertools import combinations
import math
import statistics
from pyspark import SparkContext
import numpy as np
import warnings

from pprint import pprint

warnings.simplefilter("ignore")

%load_ext autotime

time: 550 µs (started: 2024-04-19 02:34:51 +00:00)


# Configuration

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/DSCI553/hw6")


class Path:
    current_dir = os.getcwd()
    data_dir = os.path.join(current_dir, "data")
    input_csv_file = os.path.join(data_dir, "hw6_clustering.txt")

    output_dir = os.path.join(current_dir, "output")
    os.makedirs(output_dir, exist_ok=True)

    task_output_file = os.path.join(output_dir, "task_op.txt")

time: 6.43 ms (started: 2024-04-19 02:35:44 +00:00)


# Task 1:  Bradley-Fayyad-Reina (BFR) algorithm
In BFR, there are three sets of points that you need to keep track of:

**Discard set (DS), Compression set (CS), Retained set (RS)**

For each cluster in the DS and CS, the cluster is summarized by:
- `N`: The number of points
- `SUM`: the sum of the coordinates of the points
- `SUMSQ`: the sum of squares of coordinates


In [32]:
import time
import numpy as np
from sklearn.cluster import KMeans


LARGE_K_FACTOR = 5


def read_data(path):
    data = []
    with open(path, "r") as f:
        for line in f:
            row = line.strip().split(",")
            data.append([float(value) for value in row])
    return np.array(data)


def split_data(arr: np.array):
    """
    Splits the input array into 5 roughly equal parts after shuffling.
    """
    np.random.shuffle(arr)
    return np.array_split(arr, 5)


def cluster_points(data: np.array, n_cluster: int):
    """
    Clusters the data points using KMeans algorithm.

    Args:
        data (np.array): The input data array containing points.
        n_cluster (int): The desired number of clusters.

    Returns:
        Dict[int, List[int]]: A dictionary where keys represent cluster IDs and values contain the indices of points
        assigned to each cluster.
    """
    model = KMeans(n_clusters=LARGE_K_FACTOR*n_cluster)
    model.fit(data[:, 2:])

    clusters = {}
    for idx, cid in enumerate(model.labels_):
        clusters.setdefault(cid, []).append(idx)
    return clusters


def update_dict_values(data_dict: dict, data: np.array, cluster_id: int, idx: list):
    """
    Update the values in a dictionary containing information about clusters.

    Parameters:
        data_dict (dict): The dictionary containing information about clusters.
        data (np.array): The input data array.
        cluster_id (int): The ID of the cluster being updated.
        idx (list): The indices of the points belonging to the cluster.
    """
    features = data[idx, 2:]

    n = len(idx)
    SUM = np.sum(features, axis=0)
    SUMSQ = np.sum(np.square(features), axis=0)
    centroid = SUM / n

    data_dict["params"][cluster_id] = [n, SUM, SUMSQ]
    data_dict["points"][cluster_id] = np.array(data[idx, 0]).astype(int).tolist()
    data_dict["centroids"][cluster_id] = centroid
    data_dict["distances"][cluster_id] = np.sqrt(np.subtract(SUMSQ / n, np.square(centroid)))


def write_intermediate_results(
    output_path: str, round_num: int, ds_params_dict: dict, cs_params_dict: dict, retained_set
):
    """
    Write intermediate clustering results to a file.

    Parameters:
        output_path (str): The path to the output file.
        round_num (int): The current round number of clustering.
        ds_params_dict (dict): A dictionary containing parameters for the Discard Set clusters.
        cs_params_dict (dict): A dictionary containing parameters for the Compression Set clusters.
        retained_set (set): A set containing isolated points in the Retained Set.
    """
    num_ds = sum(value[0] for value in ds_params_dict.values())
    num_cs = sum(value[0] for value in cs_params_dict.values())
    with open(output_path, "a") as f:
        if round_num == 1:
            f.write("The intermediate results:\n")

        result_str = f"Round {round_num}: {num_ds},{len(cs_params_dict)},{num_cs},{len(retained_set)}\n"
        f.write(result_str)


def BFR(data: np.array, n_cluster: int, output_path: str):
    # Step 1: Load 20% of the data randomly
    data = split_data(data)
    split1 = data[0]

    # Step 2. Run K-Means (e.g., from sklearn) with a large K (e.g., 5 times of the number of the input clusters)
    # on the data in memory using the Euclidean distance as the similarity measurement
    clusters = cluster_points(split1, n_cluster)

    # Step 3: In the K-Means result from Step 2, move all the clusters that contain only one point to RS (outliers).
    retained_set = {idx[0] for idx in clusters.values() if len(idx) == 1}
    discarded_split1 = np.delete(split1, list(retained_set), axis=0)

    # Step 4. Run K-Means again to cluster the rest of the data points with K = the number of input clusters.
    clusters = cluster_points(discarded_split1, n_cluster)

    # Create Dictionaries to keep track of data
    discarded_set_dict = {"params": {}, "centroids": {}, "distances": {}, "points": {}}
    compressed_set_dict = {"params": {}, "centroids": {}, "distances": {}, "points": {}}

    # Step 5. Use the K-Means result from Step 4 to generate the DS clusters (i.e., discard their points and
    # generate statistics).
    for cluster_id, idx in clusters.items():
        update_dict_values(discarded_set_dict, discarded_split1, cluster_id, idx)

    # The initialization of DS has finished, so far, you have K numbers of DS clusters (from Step 5) and some
    # numbers of RS (from Step 3).

    # Step 6. Run K-Means on the points in the RS with a large K (e.g., 5 times of the number of the input clusters) to
    # generate CS (clusters with more than one points) and RS (clusters with only one point).
    split1_retained = discarded_split1[list(retained_set), :]
    if len(retained_set) >= LARGE_K_FACTOR * n_cluster:
        clusters = cluster_points(split1_retained, n_cluster)
        retained_set = {idx[0] for idx in clusters.values() if len(idx) == 1}

        for cluster_id, idx in clusters.items():
            if len(idx) > 1:
                update_dict_values(compressed_set_dict, split1_retained, cluster_id, idx)

    write_intermediate_results(
        output_path, 1, discarded_set_dict["params"], compressed_set_dict["params"], retained_set
    )



def task(input_path: str, n_cluster: int, output_path: str):
    try:
        start_time = time.time()

        data = read_data(input_path)

        BFR(data, n_cluster, output_path)

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    except Exception as e:
        print(e)


# if __name__ == "__main__":
#     # Check if correct number of command-line arguments are provided
#     if len(sys.argv) != 4:
#         print("Usage: python task3.py <input_filename> <stream_size> <num_of_asks> <output_filename>")
#         sys.exit(1)

#     # Parse command-line arguments
#     input_path = sys.argv[1]
#     n_cluster = int(sys.argv[2])
#     output_path = sys.argv[3]

#     # Call task1 function
#     task3(input_path, n_cluster, output_path)

task(Path.input_csv_file, 5, Path.task_output_file)

Duration: 11.409284830093384

time: 11.4 s (started: 2024-04-19 09:13:10 +00:00)


In [33]:
%cat output/task_op.txt

The intermediate results:
Round 1: 64463,0,0,0
time: 122 ms (started: 2024-04-19 09:27:58 +00:00)


# THE END